We have done several kinds of DEMs:

- All-instrument DEMs (all - either AIA+NuSTAR, or AIA+NuSTAR+XRT, depending on XRT availability)
- no-XRT DEMs (for XRT intervals)
- AIA + XRT DEMs (for XRT intervals)
- only AIA DEMs (all)

We want to compare the relative effects of each combination of instruments. As such, we would like to compare:

- AIA only vs. AIA+NuSTAR DEMs (for ALL cases)
- AIA only vs. AIA+NuSTAR vs. AIA+NuSTAR+XRT vs. AIA+XRT (for XRT-having DEMs)

We are just going to do this with the full temperature range (logT = 5.6-7.2). 

Parameters of interest to compare are:

- Emission Measure >5 MK, >7 MK, >10 MK

In [2]:
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import visualize_dem_results as viz
import all_nu_analysis as ana

import pickle
import os

import numpy as np

import matplotlib
from matplotlib import pyplot as plt

colors_list = np.concatenate((matplotlib.cm.tab10(range(10)), matplotlib.cm.Dark2(range(10))), axis=0)




In [ ]:
# #FOR WHEN WE REMOVE FILES

# os.remove(ff)

In [ ]:
flare_res = ana.get_saved_flares(add_stdv_flares=True, add_manual_flares=True)
early_starts = flare_res[0]
late_stops = flare_res[1]


In [ ]:
#AIA-only fileslist

aiachanax = ['A94', 'A131', 'A171', 'A193', 'A211', 'A335']
aianuchanax = ['A94', 'A131', 'A171', 'A193', 'A211', 'A335', '2.5-3.5keV A+B', '3.5-6.0keV A+B', '6.0-10.0keV A+B']

def dofile(r, testchanax=aiachanax, lenrange=[6,6]):

    res = viz.get_DEM_params(r)
    if not res:
        return

    m1, max1, above5_, above7_, above10_, \
        above_peak, below_peak, above_635, below_635, \
           chanax, dn_in, edn_in, \
                powerlaws, EMT_all, EMT_thresh = res

    #print(chanax)

    # if chanax != testchanax:
    #     if len(chanax) == len(testchanax):
    #         print('not the same, but the same length:')
    #         print(chanax)
    #         print(testchanax)
            
    #     print('who are you and what did you do to my aia only dem')
    #     #print(r)
    #     return


    if np.logical_and(len(chanax) >= lenrange[0], len(chanax) <= lenrange[1]):
        
        data, timestring, time = viz.load_DEM(r)
        flare = ana.check_for_flare(time, early_starts, late_stops)
        #print(flare)
        return above10_, flare, time

    else:
        #print('Length of channels list not in range: ', lenrange)
        #print(chanax)
        return



with open('all_targets.pickle', 'rb') as f:
    keydict = pickle.load(f)

keys = keydict.keys()
#keys = ['01-sep-15', '02-sep-15', '19-feb-16'] #, '22-apr-16_1']

dictz = {'all regions': {'flare files': [],
                       'quiet files': []}
        }


for key in keys:
    conditions = ['onlyaia', 'aiaxrt', 'no_xrt', key+'_MC']
    #conditions = ['onlyaia', 'no_xrt']
    lenranges = [[6,6], [7,9], [9,9], [9,12]]

    ardict = keydict[key]
    dictz[key+' region_0'] = {'key': key}
    if len(ardict['loc']) == 2:
        dictz[key+' region_1'] = {'key': key}
        
        
    
    for c in range(0, len(conditions)):
        cc = conditions[c]
        if cc == 'onlyaia':
            testchanax = aiachanax
        if cc == 'no_xrt': 
            testchanax = aianuchanax
        if cc in ['aiaxrt', key+'_MC']:
            testchanax = []

        flarefiles = []
        quietfiles = []
        flaretimes = []
        quiettimes = []
        quietxrttimes = []
        flarexrttimes = []
            
        res_files_oa, tworegions = ana.get_key_resultfiles(key, fromhome=True, 
                            keydict=keydict,
                            withparams=False,
                            namesearchstring=cc,
                            shush=True)

        if c == 3:
            cc = 'all-inst'
        
        if tworegions:
            for j in range(0,2):
                flarefiles = []
                quietfiles = []
                flaretimes = []
                quiettimes = []
                quietxrttimes = []
                flarexrttimes = []
                reglab = 'region_'+str(j)
                #label = ARDict['NOAA_ARID'][j]+'-'+key
                resfiles = res_files_oa[j]
                for r in resfiles:
                    #print(r)
                    res = dofile(r, testchanax=testchanax, lenrange=lenranges[c])
                    if res is not None:
                        a10, flare, time = res
        
                        if flare:
                            flarefiles.append(r.split('.p')[-2]+'_withparams.pickle')
                            flaretimes.append(time)
                        else:
                            #print(r)
                            quietfiles.append(r.split('.p')[-2]+'_withparams.pickle')
                            quiettimes.append(time)

                        if c == 1:
                            if flare:
                                flarexrttimes.append(time)
                            else:
                                quietxrttimes.append(time)
                    
    
                dictz[key+' '+reglab]['flare files '+cc] = flarefiles
                dictz[key+' '+reglab]['quiet files '+cc] = quietfiles
                dictz[key+' '+reglab]['flare times '] = flaretimes
                dictz[key+' '+reglab]['quiet times '] = quiettimes
                if c == 1:
                    dictz[key+' '+reglab]['flare xrt times '] = flarexrttimes
                    dictz[key+' '+reglab]['quiet xrt times '] = quietxrttimes

                if c == 3:
                    dictz['all regions']['flare files'].extend(flarefiles)
                    dictz['all regions']['quiet files'].extend(quietfiles)
                                         
        
        else:
            resfiles = res_files_oa
            reglab = 'region_0'
            for r in resfiles:
                #print(r)
                res = dofile(r, testchanax=testchanax, lenrange=lenranges[c])
                if res is not None:
                    a10, flare, time = res
    
                    if flare:
                        flarefiles.append(r.split('.p')[-2]+'_withparams.pickle')
                        flaretimes.append(time)
                    else:
                        #print(r)
                        quietfiles.append(r.split('.p')[-2]+'_withparams.pickle')
                        quiettimes.append(time)

                    if c == 1:
                        if flare:
                            flarexrttimes.append(time)
                        else:
                            quietxrttimes.append(time)
                
    
            dictz[key+' '+reglab]['flare files '+cc] = flarefiles
            dictz[key+' '+reglab]['quiet files '+cc] = quietfiles
            dictz[key+' '+reglab]['flare times '] = flaretimes
            dictz[key+' '+reglab]['quiet times '] = quiettimes
            if c == 1:
                dictz[key+' '+reglab]['flare xrt times '] = flarexrttimes
                dictz[key+' '+reglab]['quiet xrt times '] = quietxrttimes

            if c == 3:
                    dictz['all regions']['flare files'].extend(flarefiles)
                    dictz['all regions']['quiet files'].extend(quietfiles)

print(dictz.keys())

In [ ]:
with open('result_file_dict.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dictz, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open('result_file_dict.pickle', 'rb') as f:
    dictz = pickle.load(f)



def print_stats(dictt):

    print(dictt['key'])
    print('Flare-time only-AIA DEMS: ', len(dictt['flare files onlyaia']))
    print('Quiet-time only-AIA DEMS: ', len(dictt['quiet files onlyaia']))
    print('Flare-time AIA+XRT DEMS: ', len(dictt['flare files aiaxrt']))
    print('Quiet-time AIA+XRT DEMS: ', len(dictt['quiet files aiaxrt']))    
    print('Flare-time AIA+NuSTAR DEMS: ', len(dictt['flare files no_xrt']))
    print('Quiet-time AIA+NuSTAR DEMS: ', len(dictt['quiet files no_xrt'])) 
    print('Flare-time All-instrument DEMS: ', len(dictt['flare files all-inst']))
    print('Quiet-time All-instrument DEMS: ', len(dictt['quiet files all-inst']))  

    totxrt = len(dictt['flare files aiaxrt']) + len(dictt['quiet files aiaxrt'])
    totany = len(dictt['flare files all-inst']) + len(dictt['quiet files all-inst'])

    #print('flare xrt times ', len(dictt['flare xrt times ']))
    #print('quiet xrt times ', len(dictt['quiet xrt times ']))
    print('')


    return (totany > 0), (totxrt > 0)

nones = []
somes = []

xrts = []
nxrts = []
    
for k in dictz.keys():
    print(k)
    if k != 'all regions':
        any, xrt = print_stats(dictz[k])
    
        if not any:
            nones.append(k)
        else:
            somes.append(k)
            if not xrt:
                nxrts.append(k)
            else:
                xrts.append(k)


print('No-result regions: ', nones)
print('Result regions: ', somes)
print('')

print('XRT regions: ', xrts)
print('No-XRT regions: ', nxrts)
print('')

all regions
01-sep-15 region_0
01-sep-15
Flare-time only-AIA DEMS:  39
Quiet-time only-AIA DEMS:  8
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-time AIA+NuSTAR DEMS:  0
Quiet-time AIA+NuSTAR DEMS:  0
Flare-time All-instrument DEMS:  39
Quiet-time All-instrument DEMS:  8

02-sep-15 region_0
02-sep-15
Flare-time only-AIA DEMS:  15
Quiet-time only-AIA DEMS:  6
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-time AIA+NuSTAR DEMS:  0
Quiet-time AIA+NuSTAR DEMS:  0
Flare-time All-instrument DEMS:  15
Quiet-time All-instrument DEMS:  6

02-sep-15 region_1
02-sep-15
Flare-time only-AIA DEMS:  35
Quiet-time only-AIA DEMS:  4
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-time AIA+NuSTAR DEMS:  0
Quiet-time AIA+NuSTAR DEMS:  0
Flare-time All-instrument DEMS:  35
Quiet-time All-instrument DEMS:  4

19-feb-16 region_0
19-feb-16
Flare-time only-AIA DEMS:  15
Quiet-time only-AIA DEMS:  10
Flare-time AIA+XRT DEMS:  0
Quiet-time AIA+XRT DEMS:  0
Flare-tim

In [6]:
len(dictz['all regions']['quiet files'])

907

In [ ]:
#WE NEED A FUNCTION WHICH TAKES A LIST OF TIMES FOR A GIVEN KEY AND FETCHES ALL THE DEM RESULTS OF A GIVEN TYPE.

def get_res_4_times(times, dir_, namesearchstring, withparams=False, returndatas=False):

    import glob

    files_ = []
    for tt in times:
        timestring = viz.make_timestring(tt)
        if withparams:
            files = glob.glob(dir_+'/'+timestring+'/'+'*5.6_7.2*'+namesearchstring+'*withparams.pickle')[0]
        else:
            #print(dir_+'/'+timestring+'/'+'*5.6_7.2*'+namesearchstring+'*.pickle')
            fs = glob.glob(dir_+'/'+timestring+'/'+'*5.6_7.2*'+namesearchstring+'*.pickle')
            files = [f for f in fs if 'withparams' not in f]

        files_.append(files)


    #print(files_)

    if not returndatas:
        return files_
    else:
        datas = [viz.load_DEM(f) for f in files_]
        return datas


#get_res_4_times(times, dir_, namesearchstring, withparams=False)




baddies=0


with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)

import pathlib
from matplotlib import pyplot as plt


fig, axes = plt.subplots(2, 1, figsize=(15,16), tight_layout = {'pad': 1}, sharex=True)


num=3
for xr in xrts:
    dictt = dictz[xr]
    #print(dictt.keys())
    # aiaxrt = dict['quiet files aiaxrt']
    # print(aiaxrt)
    # print('')


    ARDict = all_targets[xr.split(' ')[0]]
    #print(ARDict)
    working_dir = ARDict['working_dir']
    method = ARDict['method']
    #handling my stupid directory naming irregularity
    if method == 'double':
        if ARDict['gauss_stats'][0][0] == 'EW':
            if xr.split(' ')[1] == 'region_0':
                thepath = working_dir+'east/'
            else:
                thepath = working_dir+'west/'

        if ARDict['gauss_stats'][0][0] == 'SN':
            if xr.split(' ')[1] == 'region_0':
                thepath = working_dir+'south/'
            else:
                thepath = working_dir+'north/'
    else:
        #the sane normal case
        thepath = working_dir+xr.split(' ')[1]+'/'

    save_path = pathlib.Path(thepath) 
    if not save_path.exists():
        thepath = working_dir


    print(thepath)

    rtds = True

    qfilelists = []
    qfilelists.append(get_res_4_times(dictt['quiet xrt times '], thepath, 'onlyaia', withparams=True, returndatas=rtds))
    qfilelists.append(get_res_4_times(dictt['quiet xrt times '], thepath, 'aiaxrt', withparams=True, returndatas=rtds))
    qfilelists.append(get_res_4_times(dictt['quiet xrt times '], thepath, xr.split(' ')[0]+'_MC', withparams=True, returndatas=rtds))
    #print(filelists)
    #for f in qfilelists:
        #print(len(f))

    ffilelists = []
    ffilelists.append(get_res_4_times(dictt['flare xrt times '], thepath, 'onlyaia', withparams=True, returndatas=rtds))
    ffilelists.append(get_res_4_times(dictt['flare xrt times '], thepath, 'aiaxrt', withparams=True, returndatas=rtds))
    ffilelists.append(get_res_4_times(dictt['flare xrt times '], thepath, xr.split(' ')[0]+'_MC', withparams=True, returndatas=rtds))
    #print(filelists)
    #for f in ffilelists:
    #    print(len(f))


    #Filelists is a list with three entries. Each entry is a list of dictionaries. If the length of the list is 1, 
        

    above10z = np.zeros((len(ffilelists[0]), 3))
    #print(above10z)
    for i in range(0, len(ffilelists[0])):
        for j in range(0,3):
            above10z[i,j] = ffilelists[j][i][0]['above_10MK'][0]


    above10zq = np.zeros((len(qfilelists[0]), 3))
    #print(above10z)
    for i in range(0, len(qfilelists[0])):
        for j in range(0,3):
            above10zq[i,j] = qfilelists[j][i][0]['above_10MK'][0]


    #Examining the cases where aia-xrt involves MORE hot material than AIA alone – all are times with only Al-poly.
    # for i in range(0, len(ffilelists[0])):
    #     if above10z[i, 0] < above10z[i, 1]:
    #         print(above10z[i, :])
    #         print(i)
    #         print(ffilelists[1][i][0]['chanax'])
    #         print(dictt['flare xrt times '][i])
    #         baddies+=1


    #print(above10z)
    print('')
        
    
    axes[0].scatter(above10z[:,0], above10z[:,1], marker='x', color=colors_list[num])
    axes[0].scatter(above10z[:,0], above10z[:,2], marker='o', color=colors_list[num], label=xr)
    for i in range(0, len(ffilelists[0])):
        axes[0].plot([above10z[i,0], above10z[i,0]], [above10z[i,1], above10z[i,2]], linestyle='dotted', color=colors_list[num])
    


    axes[1].scatter(above10zq[:,0], above10zq[:,1], marker='x', color=colors_list[num])
    axes[1].scatter(above10zq[:,0], above10zq[:,2], marker='o', color=colors_list[num], label=xr)
    for i in range(0, len(qfilelists[0])):
        axes[1].plot([above10zq[i,0], above10zq[i,0]], [above10zq[i,1], above10zq[i,2]], linestyle='dotted', color=colors_list[num])
    

    num+=1

#print('bad Ones: ', baddies)

for ax in axes:
    ax.plot([1e18, 3e25], [1e18, 3e25], linestyle='dashed', color='black', alpha=0.5)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylim([1e17, 1e28])
    ax.set_xlim([5e24, 3e25])
    legend1 = ax.legend(ncol=3, fontsize=13)
    ax.set_ylabel('Total EM > 10 MK [cm^-5]', fontsize=13)
    ax.set_xlabel('AIA Only - Total EM > 10 MK [cm^-5]', fontsize=13)


    lab1 = ax.scatter([0,1],[0,1], marker='x', color='black', label='AIA+XRT')
    lab2 = ax.scatter([0,1],[0,1], marker='o', color='black', label='AIA+XRT+NuSTAR')

    legend2 = ax.legend([lab1, lab2], ['AIA+XRT', 'AIA+XRT+NuSTAR'], loc=4, fontsize=13)
    ax.add_artist(legend1)
    ax.add_artist(legend2)

props = dict(boxstyle='round', facecolor='white', alpha=0.5)
axes[0].text(0.05, 0.95, 'Flaring', transform=axes[0].transAxes, fontsize=16,
                            verticalalignment='top', bbox=props)
axes[1].text(0.05, 0.95, 'Quiescent', transform=axes[1].transAxes, fontsize=16,
                            verticalalignment='top', bbox=props)

plt.savefig('aia_xrt_nustar_comparison_by_flaring.png')

In [ ]:

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)

import pathlib
from matplotlib import pyplot as plt

fig, axes = plt.subplots(4, 1, figsize=(15,22), tight_layout = {'pad': 0.5}, sharex=True)


above10z_med_all = []
above10z_thin_all = []
above10z_poly_all = []
above10z_all = []



num=3
for xr in xrts:
    dictt = dictz[xr]
    #print(dictt.keys())
    # aiaxrt = dict['quiet files aiaxrt']
    # print(aiaxrt)
    # print('')


    ARDict = all_targets[xr.split(' ')[0]]
    #print(ARDict)
    working_dir = ARDict['working_dir']
    method = ARDict['method']
    #handling my stupid directory naming irregularity
    if method == 'double':
        if ARDict['gauss_stats'][0][0] == 'EW':
            if xr.split(' ')[1] == 'region_0':
                thepath = working_dir+'east/'
            else:
                thepath = working_dir+'west/'

        if ARDict['gauss_stats'][0][0] == 'SN':
            if xr.split(' ')[1] == 'region_0':
                thepath = working_dir+'south/'
            else:
                thepath = working_dir+'north/'
    else:
        #the sane normal case
        thepath = working_dir+xr.split(' ')[1]+'/'

    save_path = pathlib.Path(thepath) 
    if not save_path.exists():
        thepath = working_dir


    print(thepath)

    rtds = True

    filelists = []
    filelists.append(get_res_4_times(dictt['quiet xrt times '], thepath, 'onlyaia', withparams=True, returndatas=rtds))
    filelists.append(get_res_4_times(dictt['quiet xrt times '], thepath, 'aiaxrt', withparams=True, returndatas=rtds))
    filelists.append(get_res_4_times(dictt['quiet xrt times '], thepath, xr.split(' ')[0]+'_MC', withparams=True, returndatas=rtds))
    #print(filelists)
    #for f in filelists:
    #    print(len(f))

    #In this version, not separating by flare vs. quiet. 
    filelists[0].extend(get_res_4_times(dictt['flare xrt times '], thepath, 'onlyaia', withparams=True, returndatas=rtds))
    filelists[1].extend(get_res_4_times(dictt['flare xrt times '], thepath, 'aiaxrt', withparams=True, returndatas=rtds))
    filelists[2].extend(get_res_4_times(dictt['flare xrt times '], thepath, xr.split(' ')[0]+'_MC', withparams=True, returndatas=rtds))
    #print(filelists)
    #for f in filelists:
    #    print(len(f))


    #Filelists is a list with three entries. Each entry is a list of dictionaries. If the length of the list is 1, 
        

    above10z_med = []
    above10z_thin = []
    above10z_poly = []
    above10z = []
    
    for i in range(0, len(filelists[0])):
        if 'Be-med x2' in filelists[1][i][0]['chanax']:
            lilist=[]
            for j in range(0,3):
                lilist.append(filelists[j][i][0]['above_10MK'][0])
            above10z_med.append(lilist)
            above10z.append(lilist)
        elif 'Be-thin x2' in filelists[1][i][0]['chanax']:
            lilist=[]
            for j in range(0,3):
                lilist.append(filelists[j][i][0]['above_10MK'][0])
            above10z_thin.append(lilist)
            above10z.append(lilist)
        elif 'Al-poly x2' in filelists[1][i][0]['chanax']:
            lilist=[]
            for j in range(0,3):
                lilist.append(filelists[j][i][0]['above_10MK'][0])
            above10z_poly.append(lilist)
            above10z.append(lilist)


    rezarrs = [above10z_med, above10z_thin, above10z_poly, above10z]
    rtitles = [['Be-med', 'Be-med, Be-thin', 'Be-med, Be-thin, Al-poly'], ['Be-thin', 'Be-thin, Al-poly'] , ['Al-poly'], ['All']]
    for r in range(0, len(rezarrs)):
        if rezarrs[r]:
            rarar=np.array(rezarrs[r])
            axes[r].scatter(rarar[:,0], rarar[:,1], marker='x', color=colors_list[num])
            axes[r].scatter(rarar[:,0], rarar[:,2], marker='o', color=colors_list[num], label=xr)
            for i in range(0, rarar.shape[0]):
                axes[r].plot([rarar[i,0], rarar[i,0]], [rarar[i,1], rarar[i,2]], linestyle='dotted', color=colors_list[num])

        # place a text box in upper left in axes coords
        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        shift=0
        for lab in rtitles[r]:
            axes[r].text(0.05, 0.95-shift, lab, transform=axes[r].transAxes, fontsize=16,
                            verticalalignment='top', bbox=props)
            shift+=0.1

    
    num+=1



    above10z_med_all.extend(above10z_med)
    above10z_thin_all.extend(above10z_thin)
    above10z_poly_all.extend(above10z_poly)
    above10z_all.extend(above10z)


# #print('bad Ones: ', baddies)
first=0
for ax in axes:
    ax.plot([1e18, 3e25], [1e18, 3e25], linestyle='dashed', color='black', alpha=0.5)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylim([1e17, 1e26])
    ax.set_xlim([5e24, 3e25])
    if first==0:
        legend1 = ax.legend(ncol=2, loc=3, fontsize=13)
        first+=1
    else:
        legend1 = ax.legend(ncol=3, loc=4, fontsize=13)
    ax.set_ylabel('Total EM > 10 MK [cm^-5]', fontsize=13)

    
    lab1 = ax.scatter([0,1],[0,1], marker='x', color='black', label='AIA+XRT')
    lab2 = ax.scatter([0,1],[0,1], marker='o', color='black', label='AIA+XRT+NuSTAR')

    legend2 = ax.legend([lab1, lab2], ['AIA+XRT', 'AIA+XRT+NuSTAR'], loc=1, fontsize=13)
    ax.add_artist(legend1)
    ax.add_artist(legend2)
    
    
axes[3].set_xlabel('AIA Only - Total EM > 10 MK [cm^-5]', fontsize=13)



plt.savefig('aia_xrt_nustar_comparison_by_xrt_filter.png')

In [ ]:


def ratioz(above10z_):

    return [a[1]/a[0] for a in above10z_], [a[2]/a[0] for a in above10z_]


rat_aax, rat_aaxn = ratioz(above10z_med_all)

num=0

thebins = np.arange(0, 1, 0.01)

fig, axes = plt.subplots(1, 1, figsize=(15,3), tight_layout = {'pad': 1})
axes.hist(rat_aax, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT / AIA', alpha=0.9, linestyle='dotted')
num+=1
axes.hist(rat_aaxn, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT-NuSTAR / AIA', alpha=0.9, linestyle='dotted')

axes.set_yscale('log')
axes.legend()
axes.set_title('Be-med + friends')

rat_aax, rat_aaxn = ratioz(above10z_thin_all)

num=0

thebins = np.arange(0, 1, 0.01)

fig, axes = plt.subplots(1, 1, figsize=(15,3), tight_layout = {'pad': 1})
axes.hist(rat_aax, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT / AIA', alpha=0.9, linestyle='dotted')
num+=1
axes.hist(rat_aaxn, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT-NuSTAR / AIA', alpha=0.9, linestyle='dotted')

axes.set_yscale('log')
axes.legend()
axes.set_title('Be-thin + friend')


rat_aax, rat_aaxn = ratioz(above10z_poly_all)

num=0

thebins = np.arange(0, 1, 0.01)

fig, axes = plt.subplots(1, 1, figsize=(15,3), tight_layout = {'pad': 1})
axes.hist(rat_aax, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT / AIA', alpha=0.9, linestyle='dotted')
num+=1
axes.hist(rat_aaxn, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT-NuSTAR / AIA', alpha=0.9, linestyle='dotted')

axes.set_yscale('log')
axes.legend()
axes.set_title('Al-poly')


rat_aax, rat_aaxn = ratioz(above10z_all)

num=0

thebins = np.arange(0, 1, 0.01)

fig, axes = plt.subplots(1, 1, figsize=(15,3), tight_layout = {'pad': 1})
axes.hist(rat_aax, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT / AIA', alpha=0.9, linestyle='dotted')
num+=1
axes.hist(rat_aaxn, bins=thebins, color=colors_list[num], edgecolor='black', 
                    label='AIA-XRT-NuSTAR / AIA', alpha=0.9, linestyle='dotted')

axes.set_yscale('log')
axes.legend()
axes.set_title('All')

Categories:

- No result regions
  - Do nothing, just make note. 
- XRT regions
  - Multi-instrument comparison for all time intervals with all three DEM types.
- no-XRT regions